In [1]:
# 데이터 관련
import pandas as pd
import numpy as np

#시간 측정용
from tqdm import tqdm

# ----------------------- 한글 ----------------------- #
import MeCab
mec = MeCab.Tagger()

# 점수 게산
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# Tokenizer , Word2Vec
from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec

import collections
import os
import math

import pickle

import datetime
from datetime import timedelta

from collections import Counter

Using TensorFlow backend.


In [70]:
# ------------------------------------------------------ 해외 브랜드 이름을 인식 일부 수정함 ------------------------------------------------------ #
# ----------------------- 정규표현식 ----------------------- #
import re
is_ProductName = re.compile("[a-zA-Z0-9]+")
# ----------------------- 형태소 분석 ----------------------- #
def MCSplit(keyword):
    ret = []
    for i in keyword.split("\n"):
        app = i.split("\t")[0]
        if app != 'EOS':
            ret.append(app)
        else:
            return ret
    return ret
def subSChar(keyword):
    # 브랜드 네임들을 연결시켜 주기위해 K-Swiss -> KSwiss " " -> "" 
    ret = re.sub("[-=+,." "!@#$%^&*(\)[\]<\>{\}\'\"/_|ㄱ-ㅎ|ㅏ-ㅣ]","",keyword)
    return ret
def KoEnMorphs(keyword):
    x = (subSChar(keyword.lower())).split()
    for i in range(0,len(x)):
        for j in is_ProductName.findall(x[i]):
            x[i] = x[i].replace(j," " + j + " ")
    x = (" ".join(x)).split()
    ret = []
    for i in x:
        if is_ProductName.fullmatch(i) == None:
            ret += MCSplit(mec.parse((i)))
        else:
            ret.append(i)
    return ret

In [10]:
# ------------------------------- Data load ------------------------------- #
beh = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv")
tra = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv")
cus = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv")
cla = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv")

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# ---------------------------------------------- Women's Shoe ---------------------------------------------- #

In [38]:
pd.read_csv("./External Data/Women's shoe.csv")

,id,asins,brand,categories,colors,count,dateAdded,dateUpdated,descriptions,dimension,...,prices.sourceURLs,prices.warranty,quantities,reviews,sizes,skus,sourceURLs,upc,websiteIDs,weight
0,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
1,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
2,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
3,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
4,AVpfEqFRLJeJML431x7w,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN
5,AVpfEqFRLJeJML431x7w,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN
6,AVpfEqFRLJeJML431x7w,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,...,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN
7,AVpfFCfi1cnluZ0-dcCZ,NaN,Sutton Studio,"Women's Suits & Sets,Women's Clothing,All Wome...",Pink,NaN,2016-01-02T03:16:24Z,2016-04-06T03:01:11Z,NaN,NaN,...,http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,8,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN
8,AVpfFCfi1cnluZ0-dcCZ,NaN,Sutton Studio,"Women's Suits & Sets,Women's Clothing,All Wome...",Pink,NaN,2016-01-02T03:16:24Z,2016-04-06T03:01:11Z,NaN,NaN,...,http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,8,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN
9,AVpfBZCAilAPnD_xTN_K,NaN,LaLeela.com,"Women's Clothing,All Women's Clothing,Clothing...",NaN,NaN,2016-03-11T09:11:01Z,2016-03-25T01:13:39Z,NaN,1 in x 1 in x 1 in,...,http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,NaN,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN


In [85]:
ext_F = pd.read_csv("./External Data/Women's shoe.csv")["brand"].drop_duplicates().values.tolist()

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (25,36,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [81]:
with open('./Tokenizer/Clac_nm/Tokenizer_Clac1_59.pickle', 'rb') as handle:
    tok60 = pickle.load(handle)

In [82]:
# 외부 데이터 삽입
for i in ext_F:
    try:
        tok60.fit_on_texts(KoEnMorphs(i))
    except:
        continue

In [83]:
# 확인
tok60.word_counts

OrderedDict([('숄', 1),
             ('썸머', 2),
             ('앙상블', 2),
             ('세트', 5),
             ('로', 102),
             ('질리', 1),
             ('크리스티나', 2),
             ('앤', 25),
             ('코', 2),
             ('여성', 100),
             ('샌들', 5),
             ('쿠션', 1),
             ('감', 1),
             ('좋', 5),
             ('은', 6),
             ('화', 2),
             ('비', 5),
             ('나인', 4),
             ('매', 10),
             ('그제', 10),
             ('이', 12),
             ('톰보이', 4),
             ('비치', 2),
             ('수영복', 22),
             ('래쉬', 1),
             ('가드', 1),
             ('반바지', 12),
             ('포커스', 2),
             ('탈모', 1),
             ('샴푸', 4),
             ('미장센', 2),
             ('염색약', 2),
             ('댕기', 1),
             ('머리', 1),
             ('트러스트', 2),
             ('려', 2),
             ('삼', 1),
             ('푸', 1),
             ('bcbg', 4),
             ('원피스', 52),
             ('키이스', 1),
   

In [84]:
# 외부 데이터 저장
with open('./Tokenizer/Clac_nm/Tokenizer_Clac1_59.pickle', 'wb') as handle:
        pickle.dump(tok60, handle, protocol=pickle.HIGHEST_PROTOCOL)